<a href="https://colab.research.google.com/github/malraharsh/Awesome-Profile-README-templates/blob/master/Hyperparams_Skin_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [329]:
DO_GET_Images = True
DO_TFRecord = True

# Setup

In [330]:
#@title Importing Libraries

import numpy as np
import os

import pandas as pd
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import scipy.ndimage
from scipy import misc
from glob import glob
from scipy import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
import skimage
import imageio
import seaborn as sns
from PIL import Image
import glob
import matplotlib.pyplot as plt
import matplotlib

import os
import cv2
import requests
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
# import scikitplot as skplt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%matplotlib inline

In [331]:
# shutil.make_archive('tfrecord', 'zip', 'tfrecord')

In [332]:
# !cp tfrecord.zip /content/drive/MyDrive/Internship/tfrecord.zip

In [333]:
#@title
%mkdir -p others data

In [334]:
path_drive_data = '/content/drive/MyDrive/Internship/'
path_images = '/content/Images'
path_drive_models = '/content/drive/MyDrive/Files/dermasync/'

In [335]:
import shutil

In [336]:
%%capture
if DO_GET_Images:
    if os.path.isdir(path_images): shutil.rmtree(path_images)
    !cp -n '/content/drive/MyDrive/Internship/Images-Full-Combined.rar' /content/others/
    if not os.path.isdir('/content/Images'):
        !unrar x '/content/others/Images-Full-Combined.rar'

In [337]:
!cp -n /content/drive/MyDrive/Internship/*.csv /content/data/
# !cp -n '/content/drive/MyDrive/Internship/tfrecord.zip.zip' /content/others/

In [338]:
%%capture
if not os.path.isdir('/content/tfrecord'):
    !unzip -o '/content/others/tfrecord.zip.zip' -d 'tfrecord'

In [339]:
path_data = '/content/Images'

In [340]:
skin_labels = ['Acne', 'Alopecia', 'Chancroid', 'Contact Dermatitis', 
          'Corns and Calluses', 'Distal-Subungual-Onychomycosis', 
          'Drug Eruption', 'Folliculitis', 'Genital Warts', 
          'Guttate Psoriasis', 'Halo Nevus', 'Hemangioma',
          'Herpes Simplex', 'Ichthyosis', 'Impetigo', 'Keloid Scar', 
          'Leprosy', 'Litchen Planus', 'Litchen Simplex Chronicus', 
          'Melasma', 'Molluscum Contagiosum', 'Nummular Dermatitis',
          'Onychomycosis', 'Photocontact Dermatitis', 'Pityriasis Alba',
          'Pityriasis Rosea', 'Pityriasis Versicolor', 'Plaque Psoriasis',
          'Polymorphic Light Eruption', 'Ringworm of Body',
          'Ringworm of Face', 'Ringworm of Hands', 'Ringworm of Scalp',
          'Scabies', 'Seborrheic Dermatitis', 'Syphilis', 'Tinea Cruris',
          'Urticaria', 'Vitiligo', 'Warts']

skin_labels = ['Warts', 'Herpes Simplex', 'Acne',  'Halo Nevus']

skin_labels = ['Warts', 'Herpes Simplex', 'Guttate Psoriasis', 'Halo Nevus', 'Acne',
       'Folliculitis', 'Nummular Dermatitis', 'Hemangioma', 'Drug Eruption',
       'Melasma', 'Pityriasis Rosea', 'Tinea Cruris', 'Scabies', 'Impetigo',
       'Corns and Calluses', 'Contact Dermatitis', 'Molluscum Contagiosum',
       'Urticaria', 'Litchen Planus', 'Keloid Scar', 'Alopecia'
       'Ringworm of Body', 'Ringworm of Hands', 'Litchen Simplex Chronicus',
       'Plaque Psoriasis', 'Ichthyosis', 'Pityriasis Versicolor',
       'Seborrheic Dermatitis', 'Ringworm of Face',
       'Polymorphic Light Eruption', 'Pityriasis Alba', 'Onychomycosis',
       'Photocontact Dermatitis', 'Vitiligo', 'Ringworm of Scalp', 'Syphilis',
       'Distal-Subungual-Onychomycosis']

skin_labels = sorted(skin_labels)
print(len(skin_labels))

36


In [341]:
path_root = '/content/'
path_dest = '/content/destination'
path_train = path_root + 'train'
path_test = path_root + 'test'
path_images = 'Images/'

In [342]:
for label in os.listdir(path_images):
    if label in skin_labels: continue
    p = os.path.join(path_images, label)    
    shutil.rmtree(p)

In [343]:
path_images_csv = '/content/data/df_path_images.csv'
if os.path.isfile(path_images_csv):
    df_path_images = pd.read_csv(path_images_csv)
else:
    df_path_images = pd.DataFrame({'disease': [], 'path': []})
    for path_img in glob.glob('Images/*/*.jpg'):
        path_split = path_img.split('/')
        disease = path_split[1]
        path_img = '/'.join(path_split[1:])
        df_path_images = df_path_images.append(dict(disease=disease, path=path_img), ignore_index=True)

In [344]:
# df_path_images.disease.value_counts().keys()

In [345]:
# balance
from sklearn.utils import resample#split data into test and training sets

def upsample_data(df, by='Skin Problem'):
    # by - using by skin problem, so count is same, data balancing
    # cols - unique values in by
    cols = df[by].unique()
    max_count = max(df[by].value_counts()) # larget category
    df_up = pd.DataFrame() # upsampled
    for col in cols:
        df_col = df.loc[df[by]==col] # data of particular disease
        upsampled = resample(df_col, replace=True, n_samples=max_count, random_state=42)
        df_up = pd.concat([df_up, upsampled])
    return df_up

In [346]:
preprocess_input_image = tf.keras.applications.mobilenet.preprocess_input

# Start

In [347]:
df = df_path_images.copy()

In [348]:
df = df[df.disease.isin(skin_labels)]
df = df.reset_index()

In [349]:
cnts = df.disease.value_counts()
import math

In [350]:
### removing dis having less than n images

min_images = 0
disease_drop = cnts[cnts < min_images].index
idx_drop = df[df.disease.isin(disease_drop)].index
df.drop(index=idx_drop, inplace=True)
df_orig = df.copy()

In [351]:
skin_labels = sorted(df.disease.unique())

In [352]:
### removing more than keep num images, for class balance

def keep_n_images(keep_n, df):
    for label in skin_labels:
        idxs = df[df.disease == label].index
        idxs_drop = idxs[keep_n:]
        df = df.drop(index=idxs_drop)
    return df

In [353]:
df.disease.value_counts()

Warts                             916
Herpes Simplex                    840
Guttate Psoriasis                 753
Halo Nevus                        681
Acne                              646
Folliculitis                      629
Nummular Dermatitis               628
Hemangioma                        527
Drug Eruption                     468
Melasma                           464
Pityriasis Rosea                  443
Tinea Cruris                      441
Scabies                           438
Impetigo                          437
Corns and Calluses                431
Contact Dermatitis                428
Molluscum Contagiosum             422
Urticaria                         402
Litchen Planus                    398
Keloid Scar                       379
Ringworm of Hands                 349
Litchen Simplex Chronicus         341
Plaque Psoriasis                  339
Ichthyosis                        320
Seborrheic Dermatitis             304
Pityriasis Versicolor             304
Ringworm of 

In [354]:
df_data = df

# Image

In [355]:
# df_data = keep_n_images(250, df_data)

df_train, df_val = train_test_split(df_data, test_size=0.01,
                                    random_state=42, 
                                    stratify=df_data['disease'])

df_train = upsample_data(df_train, by='disease')

df_test = df_val

In [356]:
batch_size = max(32, len(skin_labels) * 2)
                 
num_train_samples = len(df_train)
num_test_samples = len(df_test)
train_batch_size = batch_size
test_batch_size = batch_size
IMAGE_SHAPE = image_shape = (224, 224)

In [357]:
train_steps = np.ceil(num_train_samples / train_batch_size)
test_steps = np.ceil(num_test_samples / test_batch_size)

In [358]:
def get_batches_train(shuffle=True):
    datagen_train = ImageDataGenerator(
            rotation_range=30,
            # rescale=1. / 255,
            width_shift_range=0.3,
            height_shift_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            # vertical_flip=True,
            # brightness_range=(0.9,1.1),
            fill_mode='nearest',
            # preprocessing_function= preprocess_input_image
            )

    batches_train = datagen_train.flow_from_dataframe(
                                    dataframe=df_train,
                                    directory=path_data,
                                    x_col='path',
                                    y_col='disease',
                                    # subset="training",
                                    batch_size=train_batch_size,
                                    seed=42,
                                    shuffle=shuffle,
                                    class_mode="sparse",
                                    target_size=image_shape)

    return batches_train

def get_batches_test(df_test=df_test):        
    datagen_test = ImageDataGenerator(
                                    #   rescale=1. / 255,
                                    #   preprocessing_function= preprocess_input_image
                                      )

    batches_test = datagen_test.flow_from_dataframe(
                                    dataframe=df_test,
                                    directory=path_data,
                                    x_col='path',
                                    y_col='disease',
                                    # subset="training",
                                    batch_size=test_batch_size,
                                    seed=42,
                                    shuffle=False,
                                    class_mode="sparse",
                                    target_size=image_shape)
    
    return batches_test

batches_train = get_batches_train()
batches_test = get_batches_test()

Found 31745 validated image filenames belonging to 35 classes.
Found 147 validated image filenames belonging to 35 classes.


In [359]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(batches_train.classes),
                                                 batches_train.classes)
class_weights = dict(enumerate(class_weights))

In [360]:
path_images = 'Images/'
# nb_images=len(tf.io.gfile.glob(path_images + '*/*.jpg'))
nb_images = df_data.shape[0]
print(tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

CLASSES = [file.encode() for file in skin_labels]
# nb_images=len(tf.io.gfile.glob('data/*/*.jpeg'))
SHARDS = 16
shared_size = math.ceil(1.0 * nb_images / SHARDS) # batch size of tfrecord
# print(shared_size)

TARGET_SIZE=224

path_images_train = path_images + df_train.path.values

2.4.1


# TFRecord

In [361]:
import math

def read_image_and_label(img_path):
  
  bits = tf.io.read_file(img_path)
  image = tf.image.decode_jpeg(bits)  
  image = tf.image.resize(image, [TARGET_SIZE, TARGET_SIZE])

  label = tf.strings.split(img_path, sep='/')[-2]
#   assert(len(label) > 1) # to make sure sep is correct
  return image, label

In [362]:
def compress_image(image, label):
  image = tf.cast(image, tf.uint8)
  image = tf.image.encode_jpeg(image, optimize_size=True, chroma_downsampling=False)
  return image, label

def _bytestring_feature(list_of_bytestrings):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
  return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

def _float_feature(list_of_floats): # float32
  return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))

def to_tfrecord(img_bytes, label):  
  class_num = np.argmax(np.array(CLASSES)==label) 
  feature = {
      "image": _bytestring_feature([img_bytes]), # one image in the list
      "class": _int_feature([class_num]),        # one class in the list      
  }
  return tf.train.Example(features=tf.train.Features(feature=feature))


In [363]:
dataset = tf.data.Dataset.list_files(path_images_train, seed=10000) # This also shuffles the images
dataset = dataset.map(read_image_and_label)
dataset = dataset.map(compress_image, num_parallel_calls=AUTO)
dataset = dataset.batch(shared_size) 

if DO_TFRecord or not os.path.exists('tfrecord'):
    if os.path.exists('tfrecord'): shutil.rmtree('tfrecord')
    os.mkdir('tfrecord')
    for shard, (image, label) in enumerate(dataset):
        shard_size = image.numpy().shape[0]
        filename = "tfrecord/" + "{:02d}-{}.tfrec".format(shard, shard_size)

        with tf.io.TFRecordWriter(filename) as out_file:
            for i in range(shard_size):
            #       print(label.numpy()[i])  
                example = to_tfrecord(image.numpy()[i],label.numpy()[i])
                out_file.write(example.SerializeToString())
        print("Wrote file {} containing {} records".format(filename, shard_size))

Wrote file tfrecord/00-916.tfrec containing 916 records
Wrote file tfrecord/01-916.tfrec containing 916 records
Wrote file tfrecord/02-916.tfrec containing 916 records
Wrote file tfrecord/03-916.tfrec containing 916 records
Wrote file tfrecord/04-916.tfrec containing 916 records
Wrote file tfrecord/05-916.tfrec containing 916 records
Wrote file tfrecord/06-916.tfrec containing 916 records
Wrote file tfrecord/07-916.tfrec containing 916 records
Wrote file tfrecord/08-916.tfrec containing 916 records
Wrote file tfrecord/09-916.tfrec containing 916 records
Wrote file tfrecord/10-916.tfrec containing 916 records
Wrote file tfrecord/11-916.tfrec containing 916 records
Wrote file tfrecord/12-916.tfrec containing 916 records
Wrote file tfrecord/13-916.tfrec containing 916 records
Wrote file tfrecord/14-916.tfrec containing 916 records
Wrote file tfrecord/15-916.tfrec containing 916 records
Wrote file tfrecord/16-916.tfrec containing 916 records
Wrote file tfrecord/17-916.tfrec containing 916 

In [364]:
#@title Visualization Utitlites
# numpy and matplotlib defaults
np.set_printoptions(threshold=15, linewidth=80)
# https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case, these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [365]:

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) #/ 255.0
    # image = preprocess_input_image(image)
    # image = tf.reshape(image, [TARGET_SIZE,TARGET_SIZE, 3])
    return image


def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "class": tf.io.FixedLenFeature([], tf.int64),   # shape [] means scalar
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)   
    image = decode_image(example['image']) 
    class_label = tf.cast(example['class'], tf.int32)
    # label = tf.one_hot(label, NUM_CLASSES)
    return image, class_label

def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # image = tf.image.rot90(image, k=1) # rotate 90º
    # image = tf.image.random_crop(image, (200, 200, 3))
    image = tf.image.random_brightness(image, max_delta=.3)
    # image = tf.image.random_jpeg_quality(image, 70, 100)
    return image, label

def get_batched_dataset(filenames, augment=False, shuffle=False, cache=False):
  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic = False

  dataset = tf.data.Dataset.list_files(filenames)
  dataset = dataset.with_options(option_no_order)
  dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=16, num_parallel_calls=AUTO)
  dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
  if augment:
    dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)
  if cache:
      dataset = dataset.cache() # This dataset fits in RAM
  if shuffle:
    dataset = dataset.shuffle(2048)
  dataset = dataset.repeat()
 
  dataset = dataset.batch(BATCH_SIZE) #, drop_remainder=True) # drop_remainder will be needed on TPU

  dataset = dataset.prefetch(AUTO) 
  return dataset
  

def get_training_dataset(augment=True, cache=False):
  return get_batched_dataset(training_filenames, shuffle=True, augment=augment, cache=cache)

def get_validation_dataset():
  return get_batched_dataset(validation_filenames)

AUTO = AUTOTUNE = tf.data.experimental.AUTOTUNE

In [366]:
# training_dataset = get_training_dataset()
# training_dataset = training_dataset.unbatch().batch(8)
# train_batch = iter(training_dataset)
# # run this cell again for next set of images
# display_batch_of_images(next(train_batch))

# Models

In [367]:
import glob
import os, json, random

VALIDATION_SPLIT = 0.07

BATCH_SIZE = batch_size

filenames=tf.io.gfile.glob('tfrecord/*.tfrec')

random.shuffle(filenames)
split = int(len(filenames) * VALIDATION_SPLIT)

training_filenames = filenames[split:]
validation_filenames = filenames[:split]

validation_steps = int(nb_images // len(filenames) * len(validation_filenames)) // BATCH_SIZE
steps_per_epoch = int(nb_images // len(filenames) * len(training_filenames)) // BATCH_SIZE

print(int(nb_images * (VALIDATION_SPLIT)))
assert(split >= 1 and validation_steps>=1)

1025


In [368]:
def create_base_model(k=0):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape = (224,224,3)))
    model.add(tf.keras.layers.Lambda(preprocess_input_image))
    base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3), include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    if k > 0:    
        for layer in base_model.layers[-k:]:
            layer.trainable = True   
    model.add(base_model)
    return model

def create_model():
    top = tf.keras.Sequential()
    top.add(Flatten())
    top.add(Dense(512, activation = 'relu'))
    top.add(BatchNormalization())
    top.add(Dropout(0.5))
    top.add(Dense(256, activation = 'relu'))
    top.add(BatchNormalization())
    top.add(Dropout(0.5))
    top.add(Dense(256, activation = 'relu'))
    top.add(BatchNormalization())
    top.add(Dropout(0.5))
    top.add(Dense(128, activation = 'relu'))
    top.add(BatchNormalization())
    top.add(Dropout(0.25))
    top.add(Dense(len(labels), activation = 'softmax'))

    base_model = create_base_model()
    x = base_model.output
    outputs = top(x)
    model = Model(inputs=base_model.input, outputs=outputs)
    return model

filepath = "/content/model.h5"

def get_model(params=None, lr=None):
    if params:
        model = create_model_adv(**params)
    else:
        model = create_model()

    model.compile(Adam(lr=lr), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

    checkpoint = ModelCheckpoint(filepath, monitor='sparse_categorical_accuracy', verbose=1,
                                save_best_only=True, mode='max')

    reduce_lr = ReduceLROnPlateau(monitor='sparse_categorical_accuracy', factor=0.5, patience=2,
                                verbose=1, mode='max', min_lr=0.00001)

    early_stop = tf.keras.callbacks.EarlyStopping(monitor='sparse_categorical_accuracy', patience=5)

    callbacks = [checkpoint, reduce_lr, early_stop]

    return model, callbacks
# model, callbacks = get_model()

In [369]:
# !cp /content/drive/MyDrive/Files/dermasync/model_tfr.h5 /content/model.h5
# model.load_weights('/content/model.h5')

# model = tf.keras.models.load_model('model.h5', compile=False)

# def create_model_adv(n_conv_layers, n_filters, k, n_dense_layers, n_dense_units, c_dropout, use_base=True):
#     model = tf.keras.Sequential()
#     if not use_base: model.add(tf.keras.layers.Input(shape=(224,224,3)))
#     model.add(tf.keras.layers.Lambda(preprocess_input_image))

#     for i in range(1, n_conv_layers + 1):
#         add_conv_layers(model, n_filters * i, c_dropout)

#     model.add(Flatten())
    
#     for i in range(1, n_dense_layers): # n - 1
#         add_dense_layers(model, n_dense_units, c_dropout)

#     model.add(Dense(len(skin_labels), activation = 'softmax'))
#     # print(model.summary())
#     if not use_base: return model
#     base_model = create_base_model(k)
#     x = base_model.output
#     outputs = model(x)
#     merged_model = Model(inputs=base_model.input, outputs=outputs)
#     return merged_model
# history = model.fit(batches_train,
#                      steps_per_epoch=train_steps,
#                      class_weight=class_weights,
#                      validation_data=batches_test,
#                      validation_steps=test_steps,
#                      epochs=5,
#                      # shuffle=True,#
#                      verbose=1,
#                      callbacks=callbacks,
#                      )

In [370]:
## model, callbacks = get_model(study.best_params)
# model, callbacks = get_model(lr=0.01)

# ds_train = get_training_dataset(True)
# ds_validation = get_validation_dataset()


In [371]:
# history = model.fit(ds_train,
#                 steps_per_epoch=steps_per_epoch, epochs=15,
#                 validation_data=ds_validation, 
#                 validation_steps=validation_steps,
#                 callbacks=callbacks
#                 )

In [372]:
# model.save('model.h5')
# !cp $filepath '/content/drive/MyDrive/Files/dermasync/model_tfr.h5'

In [373]:
from sklearn.metrics import precision_score, accuracy_score, confusion_matrix, classification_report 
def get_report(df=df_test, name='Report', top_k=True, k=5):
    print(f'{name}')
    batches = get_batches_test(df)
    y_true = np.array(batches.classes)
    pred_images = y_pred = y_proba = model.predict(batches, verbose=1)
    y_pred = np.argmax(y_pred, axis=1)
    label2idx = batches.class_indices
    idx2label = {v:k for k, v in label2idx.items()}
    print(classification_report(y_true, y_pred))
    if top_k:
        get_top_k_acc(k, y_true, y_proba)

# ## converting y to categorical
def get_top_k_acc(k, y_true, pred_images):
    pred_true = np.zeros((len(y_true), len(skin_labels)))
    for i, n in enumerate(y_true):
        pred_true[i][n] = 1
    for k in range(1, k+1):
        kacc = tf.keras.metrics.top_k_categorical_accuracy(pred_true, pred_images, k=k).numpy()
        acc = sum(kacc) / len(kacc)
        print(k, ' - ', acc)
        # if acc > 0.98: break

In [374]:
# get_report()

In [375]:
# batches = get_validation_dataset()
# model.evaluate(batches, steps=10)

# More

## Tsne vis

In [376]:
# def merge_batches(n, batch):
#     list_X, list_y = [], []
#     for _ in range(n):
#         X, y = next(batch)
#         X = X.reshape(X.shape[0], -1)
#         list_X.append(X)
#         y = [np.argmax(i) for i in y]
#         list_y.extend(y)
#     return np.concatenate(list_X), list_y

In [377]:
# m = {idx: val for val, idx in batches_train.class_indices.items()}

In [378]:
# X, y = merge_batches(30, batches_train)
# X.shape, len(y)

In [379]:
# X, y = pd.DataFrame(X), pd.DataFrame(y)
# df = X
# df['label'] = y

In [380]:
# import time

# from sklearn.manifold import TSNE
# from cuml.manifold import TSNE
# #n_sne = 7000

# time_start = time.time()
# tsne = TSNE(n_iter=10)
# tsne_results = tsne.fit_transform(df.values)

# print ('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))



In [381]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# # Create the figure
# fig = plt.figure( figsize=(8,8) )
# ax = fig.add_subplot(1, 1, 1, title='TSNE' )
# # Create the scatter
# ax.scatter(
#     x=tsne_results[:,0], 
#     y=tsne_results[:,1], 
#     c=df['label'], 
#     cmap=plt.cm.get_cmap('Paired'), 
#     alpha=0.4)
# plt.show()

In [382]:
## not working
### model tf 2.4 to tf2.1 for pc

# %%capture
# !pip3 install 
# !virtualenv env
# pathenv = '/content/env/bin/activate'

# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("modelweight.h5")
# print("Saved model to disk")

In [383]:
# %%capture
# !pip3 install tensorflow==2.1
# model_pc = get_model()
# model_pc.load_weights('model.h5')
# model_pc.save('model_pc.h5')

# from google.colab import files
# files.download('model_pc.h5')

# Start

In [384]:
!pip install --quiet optuna
from tensorflow.keras.layers import Conv2D
import warnings
warnings.filterwarnings(action='once')

In [385]:
ds_train = get_training_dataset(cache=False)
ds_validation = get_validation_dataset()

In [386]:
def add_conv_layers(model, filter, dropout, n_layers=2): # n_layers: cont layers in stack, c: choice
    for i in range(n_layers - 1):
        model.add(Conv2D(filters = filter, kernel_size = 3, padding = 'same', activation = 'relu'))
        model.add(BatchNormalization())
    model.add(Conv2D(filters = filter, kernel_size = 5, strides = 2, padding = 'same', activation = 'relu'))
    model.add(BatchNormalization())
    # model.add(MaxPool2D(pool_size = (2,2))) # added stride 2 instead
    model.add(Dropout(dropout))

def add_dense_layers(model, units, dropout):
    model.add(Dense(units, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))

In [397]:
History = []

def get_model_adv(params_model):
    model = create_model_adv(**params_model)

    model.compile(Adam(lr=params_model['c_lr']), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

    checkpoint = ModelCheckpoint(filepath, monitor='sparse_categorical_accuracy', verbose=1,
                                save_best_only=True, mode='max')

    reduce_lr = ReduceLROnPlateau(monitor='sparse_categorical_accuracy', factor=0.5, patience=2,
                                verbose=1, mode='max', min_lr=0.00001)

    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=3)

    callbacks = [reduce_lr, early_stop] # checkpoint

    return model, callbacks


def create_model_adv(n_conv_layers, n_filters, n_dense_layers, n_dense_units, c_dropout, n_base_layers, **_):
    model = tf.keras.Sequential()

    for i in range(1, n_conv_layers + 1):
        add_conv_layers(model, n_filters * i, c_dropout)

    model.add(Flatten())
    
    for i in range(1, n_dense_layers): # n - 1
        add_dense_layers(model, n_dense_units, c_dropout)

    model.add(Dense(len(skin_labels), activation = 'softmax'))
    # n_base_layers
    base_model = create_base_model(n_base_layers)
    # x = base_model.output
    # outputs = model(x)
    merged_model = Sequential([base_model,
                            #    tf.keras.layers.GlobalAveragePooling2D(),
                               model])
    return merged_model

def objective(trial):
    joblib.dump(study, 'study.pkl')

    n_conv_layers = trial.suggest_categorical('n_conv_layers', [0])
    n_filters = trial.suggest_categorical('n_filters', [0])

    n_dense_layers = trial.suggest_categorical('n_dense_layers', [2, 3, 4]) # if not skip_optuna else params_best['n_dense_layers']
    n_dense_units = trial.suggest_categorical('n_dense_units', [256, 512, 1024]) #if not skip_optuna else params_best['n_dense_units']

    n_base_layers = trial.suggest_int('n_base_layers', 0, 6, 2) # if not skip_optuna else params_best['n_base_layers']

    c_dropout = trial.suggest_categorical('c_dropout', [0, 0.3, 0.5]) if not skip_optuna else params_best['c_dropout']
    c_lr = trial.suggest_loguniform('c_lr', 0.0001, 0.01) if not skip_optuna else params_best['c_lr']

    params_model = dict(
        n_conv_layers = n_conv_layers,
        n_filters = n_filters,
        n_dense_layers = n_dense_layers,
        n_dense_units = n_dense_units,
        c_dropout = c_dropout,
        n_base_layers = n_base_layers,
        c_lr = c_lr,
    )

    pruning_callback = optuna.integration.TFKerasPruningCallback(trial, 'val_sparse_categorical_accuracy')

    model, callbacks = get_model_adv(params_model)
    callbacks += [pruning_callback]
    # ds_train = get_training_dataset()
    # ds_validation = get_validation_dataset()

    history = model.fit(ds_train,
                    steps_per_epoch=steps_per_epoch, epochs=8,
                    validation_data=ds_validation, 
                    validation_steps=validation_steps,
                    callbacks=callbacks,
                    verbose=1)  
      
    History.append(history)
    return history.history['val_sparse_categorical_accuracy'][-1]

In [388]:
import optuna 
import joblib

skip_optuna = False

if os.path.isfile('study.pkl'):
    study = joblib.load('study.pkl')
else:
    study = optuna.create_study(sampler=optuna.samplers.TPESampler(multivariate=True), direction='maximize')

## set values you know well is good
# params_best=  {'n_conv_layers': 0, 'n_filters': 0, 'n_dense_layers': 3, 'n_dense_units': 2048, 'n_base_layers': 2, 'c_dropout': 0.5, 'c_lr': 0.005}

# study.enqueue_trial(params_best)

# study.optimize(objective, n_trials=25, timeout=3600)

In [389]:
from tensorflow.keras import Sequential

In [390]:
# !cp study.pkl $path_drive_data
# df = study.trials_dataframe()
# df.head(3)

In [391]:
trial = study.best_trial
params_best = trial.params
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.6666666865348816
Best hyperparameters: {'n_conv_layers': 0, 'n_filters': 0, 'n_dense_layers': 2, 'n_dense_units': 512, 'n_base_layers': 2, 'c_dropout': 0.5, 'c_lr': 0.00016699936123254085}


In [392]:
# optuna.visualization.plot_optimization_history(study)

In [393]:
# optuna.visualization.plot_slice(study)

In [398]:
params_best=  {'n_conv_layers': 0, 'n_filters': 0, 'n_dense_layers': 3, 'n_dense_units': 1024, 'n_base_layers': 0, 'c_dropout': 0, 'c_lr': 0.001}
print(params_best)
# model, _ = get_model_adv(params_best)

{'n_conv_layers': 0, 'n_filters': 0, 'n_dense_layers': 3, 'n_dense_units': 1024, 'n_base_layers': 0, 'c_dropout': 0, 'c_lr': 0.001}


In [400]:
# model = tf.keras.models.load_model('model_36class.h5')

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='sparse_categorical_accuracy', factor=0.5, patience=2,
                            verbose=1, mode='max', min_lr=0.00001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=5)

callbacks = [reduce_lr, early_stop]



history = model.fit(ds_train,
                steps_per_epoch=steps_per_epoch,
                epochs=25,
                validation_data=ds_validation, 
                validation_steps=validation_steps,
                callbacks=callbacks,
                verbose=1) 

Epoch 1/25
197/197 [==============================] - 42s 195ms/step - loss: 1.7945 - sparse_categorical_accuracy: 0.6066 - val_loss: 1.3080 - val_sparse_categorical_accuracy: 0.6623
Epoch 2/25
197/197 [==============================] - 38s 191ms/step - loss: 1.1737 - sparse_categorical_accuracy: 0.7015 - val_loss: 1.0399 - val_sparse_categorical_accuracy: 0.7390
Epoch 3/25
197/197 [==============================] - 38s 191ms/step - loss: 0.7889 - sparse_categorical_accuracy: 0.7920 - val_loss: 0.8235 - val_sparse_categorical_accuracy: 0.7974
Epoch 4/25
197/197 [==============================] - 38s 191ms/step - loss: 0.6512 - sparse_categorical_accuracy: 0.8258 - val_loss: 0.7478 - val_sparse_categorical_accuracy: 0.8169
Epoch 5/25
197/197 [==============================] - 38s 191ms/step - loss: 0.4697 - sparse_categorical_accuracy: 0.8774 - val_loss: 0.6683 - val_sparse_categorical_accuracy: 0.8390
Epoch 6/25
197/197 [==============================] - 38s 191ms/step - loss: 0.4392 -

In [ ]:
model.evaluate(ds_train, steps=30)

In [ ]:
model.evaluate(ds_validation, steps=30)

In [ ]:
def get_report_tf(ds, steps, k=5):
    y_true = []
    y_proba = []
    ds_iter = iter(ds)
    for i in range(steps):
        images, labels = next(ds_iter)
        y_true.extend([x.numpy() for x in labels])
        proba = model.predict(images)
        y_proba.extend(proba)
    y_pred = np.argmax(y_proba, axis=1)
    print(classification_report(y_true, y_pred))
    if k:
        get_top_k_acc(k, y_true, y_proba)

ds = get_validation_dataset()
get_report_tf(ds, steps=validation_steps)

In [ ]:
get_report(df_data)

In [ ]:
model.save('model.h5')

In [ ]:
# !cp model_36class.h5 $path_drive_data